In [48]:
!pip install transformers sentence-transformers

In [49]:
import pandas as pd

df = pd.read_csv('anime.csv')
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [51]:
# we find missing data , we drop
print(df.isnull().sum().sum()/len(df))

df = df.dropna()

0.025784935741011875


In [52]:
df = df.assign(
    description = df['name'] + ' ' + df['genre'] + ' ' + \
    df['type'] + ' ' + df['episodes']
)
df.head()

,anime_id,name,genre,type,episodes,rating,members,description
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,"Kimi no Na wa. Drama, Romance, School, Superna..."
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,"Fullmetal Alchemist: Brotherhood Action, Adven..."
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262,"Gintama° Action, Comedy, Historical, Parody, S..."
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572,"Steins;Gate Sci-Fi, Thriller TV 24"
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266,"Gintama&#039; Action, Comedy, Historical, Paro..."


In [53]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

anime_embeddings = model.encode(df['description'].tolist())

In [56]:
from sklearn.metrics.pairwise import cosine_similarity

def get_recommendations(query, embeddings, df, top_n=5):
    query_embedding = model.encode([query])
    similarities = cosine_similarity(query_embedding, embeddings)
    top_indices = similarities[0].argsort()[-top_n:][::-1]
    return df.iloc[top_indices]

In [57]:
query = "Romantic movie for my girlfriend with rating more than 7"
recommendations = get_recommendations(query, anime_embeddings, df)
print(recommendations[['name', 'genre']])

                              name                               genre
5887  Cutey Honey Flash: The Movie   Adventure, Comedy, Magic, Romance
4850                   Palme no Ki  Adventure, Drama, Fantasy, Romance
2771                Wonderful Days      Action, Drama, Romance, Sci-Fi
354              Tamako Love Story      Comedy, Romance, Slice of Life
1571           Marmalade Boy Movie                     Romance, Shoujo


In [59]:
query = "Romantic movie for my girlfriend "
recommendations = get_recommendations(query, anime_embeddings, df)
print(recommendations[['name', 'genre']])

                                    name                               genre
2771                      Wonderful Days      Action, Drama, Romance, Sci-Fi
4850                         Palme no Ki  Adventure, Drama, Fantasy, Romance
1376  Date A Live Movie: Mayuri Judgment              Harem, Romance, Sci-Fi
5887        Cutey Honey Flash: The Movie   Adventure, Comedy, Magic, Romance
1571                 Marmalade Boy Movie                     Romance, Shoujo


In [58]:
query = "romance funny anime movies"
recommendations = get_recommendations(query, anime_embeddings, df,5)
print(recommendations[['name', 'genre','type']])

                                    name                        genre   type
10123                            Samurai               Drama, Romance  Movie
5787              Mainichi ga Nichiyoubi              Comedy, Romance    OVA
282                       Lovely★Complex      Comedy, Romance, Shoujo     TV
375    Sekaiichi Hatsukoi: Valentine-hen  Comedy, Romance, Shounen Ai  Movie
4761                             Futakoi      Comedy, Romance, School     TV


## New Part , New Data

I will be scraping and formulating the anime.csv such that new data is recommended

In [ ]:
base_url = "https://myanimelist.net/topanime.php"

df = pd.read_html(base_url)[0]
df = df.iloc[: , 1:-2]
df.head()